In [19]:
import os
import re

import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import anndata

#### integrated object cell type refinement

In [7]:
# raw counts are raw.X
counts_adata = sc.read_h5ad('../data/single_cell/original/integrated_rna.h5ad')
counts_adata

AnnData object with n_obs × n_vars = 113052 × 31915
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.0.75', 'seurat_clusters', 'sample_id', 'cell_type'
    var: '_index', 'features'
    obsm: 'X_umap'

In [9]:
set(counts_adata.raw.X[0].toarray().flatten())

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 29.0,
 31.0,
 33.0,
 34.0,
 41.0,
 52.0,
 58.0,
 67.0}

In [17]:
counts_adata.var

,_index,features
0,AL627309.1,AL627309.1
1,AL669831.5,AL669831.5
2,LINC00115,LINC00115
3,SAMD11,SAMD11
4,NOC2L,NOC2L
...,...,...
31910,RP11-403B2.6,RP11-403B2.6
31911,RP11-1136G4.1,RP11-1136G4.1
31912,AP005530.1,AP005530.1
31913,AC008440.10,AC008440.10


In [8]:
# X is scale.data from integrated assay and raw.X is data from integrated assay
adata = sc.read_h5ad('../data/single_cell/original/integrated_integrated.h5ad')
adata

/data/estorrs/miniconda3/envs/sc_pdac/lib/python3.7/site-packages/anndata/compat/__init__.py:235: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,


AnnData object with n_obs × n_vars = 113052 × 3000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.0.75', 'seurat_clusters', 'sample_id', 'cell_type'
    var: 'SCT_features', '_index', 'features'
    uns: 'neighbors'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'SCT'
    obsp: 'distances'

In [14]:
adata.var

,SCT_features,_index,features
0,SAMD11,SPP1,SPP1
1,HES4,COL1A1,COL1A1
2,ISG15,LUM,LUM
3,TNFRSF18,MGP,MGP
4,TNFRSF4,SFRP2,SFRP2
...,...,...,...
2995,TP53TG3D,LINC00205,LINC00205
2996,CSF3,SYT11,SYT11
2997,PTPRT,BCL2L10,BCL2L10
2998,KCNB1,ATP6V1G2,ATP6V1G2


In [16]:
adata.layers['SCT'].shape

(113052, 3000)

In [13]:
adata.raw.X.shape

(113052, 3000)

In [11]:
adata.layers['counts'] = counts_adata.raw.X

ValueError: Value passed for key 'counts' is of incorrect shape. Values of layers must match dimensions (0, 1) of parent. Value had shape (113052, 31915) while it should have had (113052, 3000).

In [3]:
adata.X

array([[-2.21198275, -2.44697471, -2.67837706, ..., -0.09416235,
        -0.07340713, -0.23156486],
       [-3.17365415, -2.19511218, -2.96876128, ..., -0.11958864,
        -0.1289242 , -0.59067   ],
       [-1.80783225, -1.67735685, -1.53501979, ..., -0.03703795,
        -0.31251288, -0.13506925],
       ...,
       [-1.92023382, 10.        , 10.        , ..., -0.09778932,
        -0.06233593, -0.82954187],
       [ 0.07365613, -2.21592259, -2.53493226, ...,  0.02935523,
        -0.18326476,  1.42124371],
       [-1.93877699, -2.70065107, -2.61501048, ..., -0.05858634,
        -0.06076754,  1.68742749]])

In [4]:
adata.layers['SCT']

array([[-0.07365578, -0.28201198, -0.64118657, ..., -0.01079224,
        -0.02593469, -0.00852385],
       [-0.15699102,  3.11789246, -0.67812708, ..., -0.04953781,
        -0.06826268, -0.04781286],
       [-0.03337659, -0.10653823, -0.36900279, ...,  0.00374125,
        -0.00983258,  0.00586419],
       ...,
       [-0.22245112,  0.54281014, -0.38986595, ..., -0.18769368,
        -0.09947098, -0.10806374],
       [-0.24002901, -0.83723332,  0.27987207, ..., -0.19848254,
        -0.10516399, -0.11461643],
       [-0.2755587 , -0.2214901 , -1.56272384, ..., -0.22003168,
        -0.11663569, -0.1277709 ]])

In [5]:
adata.raw.X

<113052x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 339156000 stored elements in Compressed Sparse Row format>

In [6]:
adata.raw.X[0].toarray()

array([[-2.54102635, -3.36626981, -3.49649925, ..., -0.06412523,
        -0.0756337 , -0.36814463]])

In [25]:
adata.var = adata.var.set_index('features')

,SCT_features,_index
features,,
SPP1,SAMD11,SPP1
COL1A1,HES4,COL1A1
LUM,ISG15,LUM
MGP,TNFRSF18,MGP
SFRP2,TNFRSF4,SFRP2
...,...,...
LINC00205,TP53TG3D,LINC00205
SYT11,CSF3,SYT11
BCL2L10,PTPRT,BCL2L10


In [29]:
sc.tl.rank_genes_groups(adata, groupby='cell_type', groups=['UNK_immune'], use_raw=False)

/data/estorrs/miniconda3/envs/sc_pdac/lib/python3.7/site-packages/scanpy/tools/_rank_genes_groups.py:421: RuntimeWarning: invalid value encountered in log2
  foldchanges[global_indices]


In [30]:
adata.uns['rank_genes_groups']

{'params': {'groupby': 'cell_type',
  'reference': 'rest',
  'method': 't-test',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'names': rec.array([('HMGB2',), ('STMN1',), ('CORO1A',), ..., ('SERPINA1',),
            ('S100A6',), ('C19orf33',)],
           dtype=[('UNK_immune', 'O')]),
 'scores': rec.array([( 108.40344,), (  79.76527,), (  53.02902,), ...,
            (-100.6699 ,), (-104.77608,), (-107.68163,)],
           dtype=[('UNK_immune', '<f4')]),
 'pvals': rec.array([(0.0000000e+000,), (1.0673687e-285,), (9.1856467e-207,), ...,
            (0.0000000e+000,), (0.0000000e+000,), (0.0000000e+000,)],
           dtype=[('UNK_immune', '<f8')]),
 'pvals_adj': rec.array([(0.00000000e+000,), (3.07894816e-284,), (1.46579469e-205,),
            ..., (0.00000000e+000,), (0.00000000e+000,),
            (0.00000000e+000,)],
           dtype=[('UNK_immune', '<f8')]),
 'logfoldchanges': rec.array([(      nan,), (      nan,), (      nan,), ..., (1.7532507,),
    